<a href="https://colab.research.google.com/github/Shriansh16/LLM_Engineering/blob/main/14_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import gradio as gr
from langchain.document_loaders import PyPDFLoader

In [8]:

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [27]:
MODEL = "gpt-3.5-turbo"
db_name = "vector_db"

In [10]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', '')

In [15]:
pdf_path='/content/ok'
loader = DirectoryLoader(pdf_path, glob='*.pdf', loader_cls=PyPDFLoader)
document = loader.load()

In [17]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(document)

In [18]:
len(chunks)

19

In [20]:
embeddings = OpenAIEmbeddings()

In [21]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 19 documents


#Time to use LangChain to bring it all together

In [22]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

<ipython-input-22-51939673df05>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [24]:
query = "what si smartcookie"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

Smart Cookie is a teacher-student reward program designed to foster a fun, interactive, and rewarding environment for both teachers and students. The program allows teachers to recognize and reward students for their achievements in various activities, such as sports, drawing, and academic tests. It aims to make the educational process more engaging and motivating by providing real-time rewards that acknowledge students' efforts and accomplishments. It was founded by Avinash Kulkarni.


###Now we will bring this up in Gradio using the Chat interface -

In [25]:
# Wrapping that in a function

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [26]:
view = gr.ChatInterface(chat).launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://75778230e9fc56775f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


###BEHIND THE SCENE

In [28]:
# Let's investigate what gets sent behind the scenes

from langchain_core.callbacks import StdOutCallbackHandler

llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

retriever = vectorstore.as_retriever()

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, callbacks=[StdOutCallbackHandler()])

query = "Who received the prestigious IIOTY award in 2023?"
result = conversation_chain.invoke({"question": query})
answer = result["answer"]
print("\nAnswer:", answer)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
1. Recognition for Achievements:  
o Real-Time Rewards: Students receive rewards instantly for their 
accomplishments, reinforcing positive behavior and achievements.  
o Mutual Recognition: Teachers and students can recognize and reward each 
other's contributions, fostering a positive environment.  
2. Extension Possibilities:  
o The system can be adapted for other settings, such as player -coach 
relationships in sports or employee -manager dynamics in workplaces, making 
it versatile.  
3. Addressing Lack of Appreciation:  
o Often, exceptional performances by students and teachers go unnoticed. 
SmartCookie aims to address this by ensuring 